# TrekList
## Build databases

This python notebook builds the base TrekList database of all series, episodes, and movies, followed by the structure of the user database. Sometimes the OMDB calls for each episode don't work, so running this multiple times to catch all episodes is sometimes necessary.

In [1]:
import omdb
import sqlite3
import pandas as pd
import os
import requests
import shutil
import io
import PIL.Image as Image

# first read in OMDB python key
with open('api_key') as f:
    lines = f.readlines()
api_key = lines[0]
omdb.set_default('apikey', api_key)

In [2]:
# all series
st_series = {'tos': 'tt0060028',
             'tas': 'tt0069637',
             'tng': 'tt0092455',
             'ds9': 'tt0106145',
             'voy': 'tt0112178',
             'ent': 'tt0244365',
             'dis': 'tt5171438',
             'sho': 'tt9059594',
             'pic': 'tt8806524',
             'lds': 'tt9184820',
             'pro': 'tt9795876',
             'snw': 'tt12327578',
             } 

## Series Request

In [3]:
db_filename = "treklist.db"
#if os.path.exists(db_filename):
#    os.remove(db_filename)

# make omdb requests for series
conn = sqlite3.connect(db_filename)
curs = conn.cursor()

conn.execute("DROP TABLE series")
conn.execute("CREATE TABLE IF NOT EXISTS series (abb TEXT, title TEXT, imdb_id TEXT, year TEXT, total_seasons INTEGER, poster_url TEXT, poster BLOB, rated TEXT)")
for key in st_series.keys():
    res = omdb.imdbid(st_series[key])

    # check to see if series already exists
    cmd = f"SELECT * from series WHERE imdb_id == '{res['imdb_id']}'"
    df = pd.read_sql_query(cmd, conn)

    # add it
    if len(df) == 0:
        cmd = "INSERT INTO series (abb, title, imdb_id, year, total_seasons, poster_url, rated) VALUES (?, ?, ?, ?, ?, ?, ?) "
        curs.execute(cmd, [key, res['title'], res['imdb_id'], res['year'], res['total_seasons'], res['poster'], res['rated']])
        conn.commit()

df = pd.read_sql_query("SELECT * FROM series", conn)
df


,abb,title,imdb_id,year,total_seasons,poster_url,poster,rated
0,tos,Star Trek,tt0060028,1966–1969,3,https://m.media-amazon.com/images/M/MV5BNDRkMT...,None,TV-PG
1,tas,Star Trek: The Animated Series,tt0069637,1973–1975,2,https://m.media-amazon.com/images/M/MV5BMzI0Y2...,None,TV-Y7
2,tng,Star Trek: The Next Generation,tt0092455,1987–1994,7,https://m.media-amazon.com/images/M/MV5BOWFhYj...,None,TV-PG
3,ds9,Star Trek: Deep Space Nine,tt0106145,1993–1999,7,https://m.media-amazon.com/images/M/MV5BMDc3OG...,None,TV-PG
4,voy,Star Trek: Voyager,tt0112178,1995–2001,7,https://m.media-amazon.com/images/M/MV5BYWIwMT...,None,TV-PG
5,ent,Star Trek: Enterprise,tt0244365,2001–2005,4,https://m.media-amazon.com/images/M/MV5BODg3Zm...,None,TV-PG
6,dis,Star Trek: Discovery,tt5171438,2017–,4,https://m.media-amazon.com/images/M/MV5BNjg1NT...,None,TV-14
7,sho,Star Trek: Short Treks,tt9059594,2018–2020,2,https://m.media-amazon.com/images/M/MV5BNWY2NW...,None,TV-PG
8,pic,Star Trek: Picard,tt8806524,2020–,3,https://m.media-amazon.com/images/M/MV5BMmMzNT...,None,TV-MA
9,lds,Star Trek: Lower Decks,tt9184820,2020–,3,https://m.media-amazon.com/images/M/MV5BNjZkNG...,None,TV-14


## Episodes Request

In [8]:
keys = list(st_series.keys())
for key in keys:
    conn.execute(f"CREATE TABLE IF NOT EXISTS {key} (title TEXT, rated TEXT, released DATE, season INTEGER, episode INTEGER, runtime TEXT, director TEXT, writer TEXT, actors TEXT, plot TEXT, poster_url TEXT, poster BLOB, imdb_rating FLOAT, imdb_votes INTEGER, imdb_id TEXT)")

    tot_seasons = df['total_seasons'][df['abb'] == key].values[0]
    print('%s seasons %i' % (key, tot_seasons))
    for season in range(tot_seasons):# another temporary....
        season += 1 # account for zero index
        #print(season)
        res = omdb.imdbid(st_series[key], season=season)
        for ep in res['episodes']:

            # check if it is already in the database
            df_check = pd.read_sql_query(f"SELECT * FROM {key} WHERE imdb_id = '{ep['imdb_id']}'", conn)
            ep_exists = True if len(df_check) > 0 else False
            if not ep_exists:
                ep_info = omdb.imdbid(ep['imdb_id'])
                cmd = f"INSERT INTO {key} (title, rated, released, season, episode, runtime, director, writer, actors, plot, poster_url, imdb_rating, imdb_votes, imdb_id) VALUES " + "(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
                curs.execute(cmd, [ep_info['title'], ep_info['rated'], ep['released'], season, ep_info['episode'], ep_info['runtime'], ep_info['director'], ep_info['writer'], ep_info['actors'], ep_info['plot'], ep_info['poster'], ep_info['imdb_rating'], ep_info['imdb_votes'], ep_info['imdb_id']])

        conn.commit()
        
# add tos - the cage
df_check = pd.read_sql_query(f"SELECT * FROM tos WHERE imdb_id = 'tt0059753'", conn)
ep_exists = True if len(df_check) > 0 else False
if not ep_exists:
    ep_info = omdb.imdbid("tt0059753")
    ep_info['season']  = 1
    ep_info['episode'] = 0
    cmd = f"INSERT INTO tos (title, rated, released, season, episode, runtime, director, writer, actors, plot, poster_url, imdb_rating, imdb_votes, imdb_id) VALUES " + "(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    curs.execute(cmd, [ep_info['title'], ep_info['rated'], ep_info['released'], ep_info['season'], ep_info['episode'], ep_info['runtime'], ep_info['director'], ep_info['writer'], ep_info['actors'], ep_info['plot'], ep_info['poster'], ep_info['imdb_rating'], ep_info['imdb_votes'], ep_info['imdb_id']])
    conn.commit()

tos seasons 3
tas seasons 2
tng seasons 7
ds9 seasons 7
voy seasons 7
ent seasons 4
dis seasons 4
sho seasons 2
pic seasons 3
lds seasons 3
pro seasons 1
snw seasons 2


## Download/Insert Posters

In [9]:
# Download posters for series only
df = pd.read_sql_query(f"SELECT * from series", conn)
for i, row in df.iterrows():
    poster_url = row['poster_url']
    if poster_url != "N/A":
        # insert poster
        img = requests.get(row['poster_url'], stream=True)
        cmd = f"UPDATE series SET poster = ? WHERE imdb_id = '{row['imdb_id']}'"
        curs.execute(cmd, [sqlite3.Binary(img.content)])
        conn.commit()

OperationalError: attempt to write a readonly database

In [ ]:
# optional - read back a poster
curs.execute(f"SELECT * FROM series WHERE imdb_id = 'tt0060028'")
record = curs.fetchall()
idx = df.keys().to_list().index('poster')
image = Image.open(io.BytesIO(record[0][idx]))
image.show()

In [ ]:
# download poster for individual episodes
for key in keys:
    print(key)
    df = pd.read_sql_query(f"SELECT * from {key}", conn)
    for i, row in df.iterrows():
        poster_url = row['poster_url']
        if poster_url != "N/A":
            # insert poster
            img = requests.get(row['poster_url'], stream=True)
            cmd = f"UPDATE {key} SET poster = ? WHERE imdb_id = '{row['imdb_id']}'"
            curs.execute(cmd, [sqlite3.Binary(img.content)])
            conn.commit()

## Movies Request

In [ ]:
# all movies
st_movies = {'TMP':  'tt0079945',
             'TWOK': 'tt0084726',
             'TSFS': 'tt0088170',
             'TVH':  'tt0092007',
             'TFF':  'tt0098382',
             'TUC':  'tt0102975',
             'GEN':  'tt0111280',
             'FC':   'tt0117731',
             'INS':  'tt0120844',
             'NEM':  'tt0253754',
             'ST09': 'tt12126100',
             'STID': 'tt1408101',
             'STB':  'tt2660888',
             }

In [ ]:
conn.execute(f"CREATE TABLE IF NOT EXISTS mov (abb TEXT, title TEXT, year TEXT, rated TEXT, released DATE, runtime TEXT, director TEXT, writer TEXT, actors TEXT, plot TEXT, poster_url TEXT, poster BLOB, metascore INTEGER, imdb_rating FLOAT, imdb_votes INTEGER, box_office TEXT, imdb_id TEXT)")

for key in st_movies.keys():
    res = omdb.imdbid(st_movies[key])

    # check if it is already in the database
    df_check = pd.read_sql_query(f"SELECT * FROM mov WHERE imdb_id = '{res['imdb_id']}'", conn)
    mov_exists = True if len(df_check) > 0 else False
    if not mov_exists:
        cmd = "INSERT INTO mov (abb, title, year, rated, released, runtime, director, writer, actors, plot, poster_url, metascore, imdb_rating, imdb_votes, imdb_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?) "
        curs.execute(cmd, [key, res['title'], res['year'], res['rated'], res['released'], res['runtime'], res['director'], res['writer'], res['actors'], res['plot'], res['poster'], res['metascore'], res['imdb_rating'], res['imdb_votes'], res['imdb_id']])
        conn.commit()
df = pd.read_sql_query("SELECT * FROM mov", conn)
df

## Download Movie Posters

In [ ]:
# Download posters for series only
df = pd.read_sql_query(f"SELECT * from mov", conn)
for i, row in df.iterrows():
    poster_url = row['poster_url']
    if poster_url != "N/A":
        # insert poster
        img = requests.get(row['poster_url'], stream=True)
        cmd = f"UPDATE mov SET poster = ? WHERE imdb_id = '{row['imdb_id']}'"
        curs.execute(cmd, [sqlite3.Binary(img.content)])
        conn.commit()

## Build User Database

In [ ]:
# delete current database if exists
db_filename = "user.db"
if os.path.exists(db_filename):
    os.remove(db_filename)

# make omdb requests for series
conn = sqlite3.connect(db_filename)
curs = conn.cursor()

conn.execute("DROP TABLE IF EXISTS log")
conn.execute("CREATE TABLE log (imdb_id TEXT, watched BOOLEAN, last_watched DATE, notes TEXT, favorite BOOLEAN, rating INTEGER, emoji TEXT)")
#conn.execute("INSERT INTO log (imdb_id, watched, last_watched) VALUES ('tt0708469', 1, '2022-02-04')")
conn.commit()

In [ ]:
df = pd.read_sql_query("SELECT * FROM log", conn)
df

In [ ]:
with open('settings.yaml') as f:
    settings = yaml.load(f, Loader=SafeLoader)
if settings['startup_window']:
    print('yo')